In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import dynamic_computation as dc
import length_priors
import numpy as np
import scipy as sp
import scipy.signal
import segment_viz
%matplotlib widget

In [ ]:
in_data = pd.read_csv('/Users/guichaoua 1/Nextcloud/Cardiac_response_to_live_music/Updated features/full_ARIs/2018_05_22_P1.csv')

In [ ]:
plt.plot(in_data.time, in_data.ARI)
plt.show()

# First try

In [ ]:
lengthPrior = length_priors.EmpiricalLengthPrior(range(5, 120), len(in_data.ARI))

In [ ]:

def make_geo_dist(min_len, max_len, ratio):
    unscaled = [ratio**i for i in range(max_len-min_len)]
    factor = sum(unscaled)
    return {i:p/factor for i, p in zip(range(min_len, max_len), unscaled)}


lengthPrior = length_priors.DiscreteLengthPrior(len(in_data.ARI), make_geo_dist(5, 60, .8), 60)

In [ ]:
est_noise = np.std(np.diff(in_data.ARI))/2

In [ ]:
arcPrior = {
    # Gaussian priors on the parameters of ax^2 + bx + c
    'aMean': 0,
    'aStd': 00,
    'bMean': 00,
    'bStd': 00,
    'cMean': 330,
    'cStd': 15,
    'noiseStd': est_noise
}

In [ ]:
post, post_bidim = dc.runAlphaBeta(in_data.ARI, arcPrior, lengthPrior, return2D=True)

In [ ]:
f=segment_viz.plot_segment_with_signal(post, in_data.ARI, in_data.time, data_color='y', boundaries=in_data.time[post>.5])

Let's plot only the strong boundaries

In [ ]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.plot(in_data.time[:len(post)], in_data.ARI[:len(post)],'r')
# plt.xlim((0,500))
ax2 = plt.twinx()

high_post = post>.5
ax2.vlines(in_data.time[high_post],0,1)
plt.show()

# Try 2D plots

In [ ]:
smoothed = sp.signal.convolve2d(post_bidim, np.ones((5,5)), mode="same")
for i in range(np.shape(smoothed)[0]):
    smoothed[i,i]=1

In [ ]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.imshow(smoothed)
plt.colorbar()

plt.show()

# Do it again with other settings

We use a Uniform Prior $U([10,90])$

In [ ]:
lengthPrior = length_priors.EmpiricalLengthPrior(range(10, 90), len(in_data.ARI))

In [ ]:
est_noise = np.std(np.diff(in_data.ARI)/2)

In [ ]:
arcPrior = {
    # Gaussian priors on the parameters of ax^2 + bx + c
    'aMean': 0,
    'aStd': 00,
    'bMean': 00,
    'bStd': 00,
    'cMean': 330,
    'cStd': 15,
    'noiseStd': est_noise
}

In [ ]:
post2, post2_bidim = dc.runAlphaBeta(in_data.ARI, arcPrior, lengthPrior, return2D=True)

In [ ]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.plot(in_data.time[:len(post)], in_data.ARI[:len(post)],'r')
# plt.xlim((500,1000))
ax2 = plt.twinx()

# ax2.plot(in_data.time[:len(post)],post)
post2_smoothed = np.convolve([1,1,1,1,1],post2,mode='same')

ax2.plot(in_data.time,post2_smoothed)
plt.show()

Let's plot only the strong boundaries

In [ ]:
fig=plt.figure(figsize=(12,8), dpi= 100)

plt.plot(in_data.time[:len(post)], in_data.ARI[:len(post)],'r')
# plt.xlim((0,500))
ax2 = plt.twinx()

high_post = post2_smoothed>.5
ax2.vlines(in_data.time[high_post],0,1)
plt.show()

In [ ]:
sum(post2)



In [ ]:
sum(post)

In [ ]:
import scipy.stats

pdf = scipy.stats.norm.pdf([0.3,0.4,-1])
pdf

# Try 2D plots

In [ ]:
segment_viz.plot_segment_matrix(post2_bidim, smoothing=5)

In [ ]:
smoothed2 = sp.signal.convolve2d(post2_bidim, np.ones((5,5)), mode="same")
for i in range(np.shape(smoothed2)[0]):
    smoothed2[i,i]=1

In [ ]:
fig, ax =plt.subplots(figsize=(12,8), dpi= 100)

im = ax.imshow(smoothed2, cmap=plt.get_cmap('hot'))
cbar = fig.colorbar(im)
cbar.set_label("Likelihood")
ax.set_ylabel('Start')
ax.set_xlabel('End')
ax.set_title('Posterior likelihood of segment')


fig.show()

In [ ]:
viz = segment_viz.plot_segment_raindrop(post2_bidim, lengthPrior.maxLength)

In [ ]:
viz2 = segment_viz.plot_segment_beams(post2_bidim, lengthPrior.maxLength)

In [ ]:
linear_viz = np.zeros((np.shape(post2_bidim)[0], lengthPrior.maxLength))
for i in range(np.shape(post2_bidim)[0]):
    for j in range(lengthPrior.maxLength):
        if i+j < np.shape(post2_bidim)[0]:
            for k in range(j):
                linear_viz[i+k,j] = linear_viz[i+k,j] + post2_bidim[i,i+j]

In [ ]:
fig=plt.figure(figsize=(16,4), dpi= 100)

plt.imshow(np.transpose(linear_viz), cmap=plt.get_cmap('hot'), origin='lower', aspect=4)
plt.colorbar()

plt.show()

In [ ]:
linear_viz_point = np.zeros((np.shape(smoothed2)[0], lengthPrior.maxLength))
for i in range(np.shape(smoothed2)[0]):
    for j in range(lengthPrior.maxLength):
        if i+j < np.shape(smoothed2)[0]:
            for k in range(1):
                linear_viz_point[i+k,j] = linear_viz_point[i+k,j] + smoothed2[i,i+j]

In [ ]:
fig=plt.figure(figsize=(16,4), dpi= 100)

plt.imshow(np.transpose(linear_viz_point), cmap=plt.get_cmap('hot'), origin='lower', aspect=4)
plt.colorbar()

plt.show()

In [ ]:
priorMarginals, prior_bidim = dc.runAlphaBeta([0 for _ in in_data.ARI], arcPrior, lengthPrior, return2D=True)

In [ ]:
_=segment_viz.plot_segment_with_signal(priorMarginals,[0 for _ in in_data.ARI],in_data.time, smoothing=1)

In [ ]:
geo_prior = length_priors.DiscreteLengthPrior(len(in_data.ARI), make_geo_dist(5, 60, .8), 60)
geo_prior_marginals, geo_prior_bidim = dc.runAlphaBeta([0 for _ in in_data.ARI], arcPrior, geo_prior, return2D=True)

In [ ]:
_=segment_viz.plot_segment_with_signal(geo_prior_marginals,[0 for _ in in_data.ARI],in_data.time, smoothing=1)

In [ ]:
uni_prior = length_priors.EmpiricalLengthPrior(list(range(10,15)),len(in_data.ARI))
uni_prior_marginals, uni_prior_bidim = dc.runAlphaBeta([0 for _ in in_data.ARI], arcPrior, uni_prior, return2D=True)

In [ ]:
_=segment_viz.plot_segment_with_signal(uni_prior_marginals,[0 for _ in in_data.ARI],in_data.time, smoothing=5)

In [ ]:
_ = segment_viz.plot_segment_raindrop(prior_bidim, max_length=120)
_ = segment_viz.plot_segment_raindrop(geo_prior_bidim, max_length=120)
_ = segment_viz.plot_segment_raindrop(uni_prior_bidim, max_length=120)

In [ ]:
print(geo_prior.distrib.get(15)**2)
print(geo_prior.distrib.get(10)*geo_prior.distrib.get(20))

In [ ]:
print(geo_prior.evalCond(-1,10) * geo_prior.evalCond(10,30))
print(geo_prior.evalCond(-1,15) * geo_prior.evalCond(15,30))

In [ ]:
geo_prior = length_priors.DiscreteLengthPrior(30, make_geo_dist(11, 18, .8), 20)
geo_prior_marginals, geo_prior_bidim = dc.runAlphaBeta([0 for _ in range(30)], arcPrior, geo_prior, return2D=True)

In [ ]:
_=segment_viz.plot_segment_with_signal(geo_prior_marginals,[0 for _ in range(30)],list(range(30)), smoothing=1)

In [ ]:
print(geo_prior_marginals)